In [1]:
from __future__ import print_function
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=4, suppress=True)
import versor as vsr
import motor_jacobian

Populating the interactive namespace from numpy and matplotlib


In [2]:
rotor = (vsr.Biv(1,0,0).unit() * pi/6).exp()
rotor0 = vsr.Biv(pi/2, 0.0, 0.0).exp()
print(rotor)

Rot: [ 0.87 -0.5 0 0 ]


In [3]:
vectors_a = [vsr.Vec(1,0,0), vsr.Vec(0,1,0), vsr.Vec(0,0,1)]
vectors_b = [a.spin(rotor) for a in vectors_a]

In [4]:
n_points=10
vectors_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)) for i in range(n_points)]
vectors_b = [vector.spin(rotor) for vector in vectors_a]

In [5]:
[a ^ b for a, b in zip()] 

[]

In [6]:
 def jacrot(a, rot):
    j = np.zeros((3,3))
    for i in range(3):
        biv = vsr.Biv(0.0, 0.0, 0.0)
        biv[i] = 1.0
        j[:,i] = 2.0 * np.array(biv.comm(a.spin(rot))) # Wrong sign?
    return j

In [7]:
_, J = motor_jacobian.diff_rotor_cost(rotor, vectors_a[0], vectors_b[0])
J

array([[ 1.5887,  0.9391,  0.1019, -0.0588],
       [ 0.9391, -1.5887,  0.0588,  0.1019],
       [ 0.1019, -0.0588, -1.5887, -0.9391]])

In [8]:
J = jacrot(vectors_a[0], rotor0)

In [9]:
def resrot(a,b,rot):
    return np.array(b - a.spin(rot)).reshape(-1,1)

In [10]:
a = vectors_a[0]
b = vectors_b[0]
J = jacrot(a, rotor0)
print(J)

[[-0.0189  0.1177  0.    ]
 [ 1.8454  0.      0.1177]
 [ 0.      1.8454  0.0189]]


In [11]:
f = resrot(a,b,rotor0)
print(f)

[[ 1.3759]
 [ 0.8133]
 [ 0.    ]]


In [12]:
np.dot(J.T,f)

array([[ 1.4748],
       [ 0.1619],
       [ 0.0957]])

In [13]:
a.spin(rotor0) ^ b

Biv: [ -0.7374 -0.08094 -0.04784 ]

In [14]:
a ^ b

Biv: [ 0.7374 0.02762 -0.04673 ]

In [15]:
np.dot()

TypeError: Required argument 'a' (pos 1) not found

In [16]:
np.dot(J.T, J)

array([[ 3.4059, -0.0022,  0.2171],
       [-0.0022,  3.4194,  0.0349],
       [ 0.2171,  0.0349,  0.0142]])

In [17]:
B

NameError: name 'B' is not defined

In [18]:
def Hess2(a,b,rot):
    H = np.zeros((3,3))
    B = a.spin(rot) ^ b
    for i in range(3):
        ei = vsr.Biv(0.0, 0.0, 0.0)
        ei[i] = 1.0
        H[:,i] = ei.comm(B)
    return H
Hess2(a,b,rotor0)

array([[ 0.    ,  0.0478, -0.0809],
       [-0.0478,  0.    ,  0.7374],
       [ 0.0809, -0.7374,  0.    ]])

In [89]:
 def Hess(a,b, rot):
    j = np.zeros((3,3))
    R = vsr.EGA(rot)
    a = vsr.EGA(a)
    b = vsr.EGA(b)
    for i in range(3):
        ei = vsr.Biv(0.0, 0.0, 0.0)
        ei[i] = 1.0
        ei = vsr.EGA(ei)
        j[:,i] = -2.0 * np.array(((ei * a * R.rev() * b - b * R * a * ei.rev())))[4:7]
    return j
np.sum([Hess(a,b,rotor0) for a,b in zip(vectors_a, vectors_b)], axis=0)

array([[-28.832 ,  -4.5848,  -5.4855],
       [  4.5848, -28.832 , -10.1947],
       [  5.4855,  10.1947, -28.832 ]])

In [26]:
j = np.zeros((3,3))
R = vsr.EGA(rotor)
a = vsr.EGA(vectors_a[0])
b = vsr.EGA(vectors_b[0])
for i in range(3):
    ei = vsr.Biv(0.0, 0.0, 0.0)
    ei[i] = 1.0
    ei = vsr.EGA(ei)
#     print(((R * a * ei.rev()) ^ b)  + ((ei * a * R.rev()) ^ b) )
    j[:,i] = -2.0 * np.array(((R * a * ei.rev() + ei * a * R.rev()) ^ b))[4:7]
j

array([[-1.1208, -0.0966, -0.3239],
       [ 0.0678, -1.2829,  0.4041],
       [-0.2847, -0.3313,  0.0092]])

In [27]:
np.array([motor_jacobian.outer_product(a, b, rotor0)[:,1:] for a,b in zip(vectors_a, vectors_b)]).reshape(1,3)

ValueError: total size of new array must be unchanged

In [28]:
motor_jacobian.outer_product(a, b, rotor0)[:,1:]

array([[ 0.    ,  0.    ,  0.    ],
       [ 1.147 , -0.4016, -0.0079],
       [ 0.6092,  1.1335,  0.5628]])

In [29]:
np.sum([motor_jacobian.outer_product(a, b, rotor0)[:,1:] for a,b in zip(vectors_a, vectors_b)], axis=0)

array([[  0.    ,   0.    ,   0.    ],
       [ 14.416 ,   0.3544,   2.0331],
       [ -2.2924,   6.1527,   3.6464]])

In [30]:
np.array([motor_jacobian.outer_product(a, b, rotor0)[:,1:] for a,b in zip(vectors_a, vectors_b)])

array([[[ 0.    ,  0.    ,  0.    ],
        [ 0.5604,  0.1644,  0.0391],
        [-0.2805,  0.4575,  0.3883]],

       [[ 0.    ,  0.    ,  0.    ],
        [ 0.5323, -0.1201, -0.6747],
        [ 0.7644, -0.0367, -0.6629]],

       [[ 0.    ,  0.    ,  0.    ],
        [ 3.0091,  0.1028,  0.2597],
        [-0.3791, -0.187 ,  1.2422]],

       [[ 0.    ,  0.    ,  0.    ],
        [ 4.329 ,  1.9326,  2.2103],
        [-4.8131,  0.6371,  1.5036]],

       [[ 0.    ,  0.    ,  0.    ],
        [ 0.4206,  0.066 , -0.0281],
        [-0.0746,  0.4435,  0.0431]],

       [[ 0.    ,  0.    ,  0.    ],
        [ 0.0153, -0.0046, -0.0054],
        [ 0.0116,  0.0021,  0.0084]],

       [[ 0.    ,  0.    ,  0.    ],
        [ 2.5853, -1.2008, -0.0222],
        [ 1.8204,  2.5569,  1.0566]],

       [[ 0.    ,  0.    ,  0.    ],
        [ 1.0434, -0.1093,  0.1106],
        [ 0.0682,  0.8168, -0.2441]],

       [[ 0.    ,  0.    ,  0.    ],
        [ 0.7736, -0.0749,  0.1517],
        [-0.019 ,  0.3

In [31]:
np.dot(J.T,J)

array([[ 1.2942, -0.2456, -0.2322],
       [-0.2456,  0.699 , -0.6461],
       [-0.2322, -0.6461,  0.7718]])

In [32]:
Hess(vectors_a[0], vectors_b[0], rotor0)

array([[-1.1208, -0.3288, -0.0783],
       [ 0.561 , -0.915 , -0.7766],
       [-0.1674, -0.306 , -0.2059]])

In [33]:
a_ = vsr.EGA(a.spin(rotor0))
b_ = vsr.EGA(b)
ei = vsr.Biv(0.0, 0.0, 0.0)
ej = vsr.Biv(0.0, 0.0, 0.0)

In [34]:
rotor0 * a *  vsr.Biv(1,0,0) 

TypeError: Incompatible function arguments. The following argument types are supported:
    1. (Rot, float) -> Rot
    2. (Rot, Biv) -> Rot
    3. (Rot, vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)0, (short)17, (short)18, (short)20> >) -> vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)0, (short)3, (short)5, (short)6, (short)17, (short)18, (short)20, (short)23> >
    4. (Rot, Rot) -> Rot


In [35]:
[np.array(-2 * a ^ b)[4:6] for 

SyntaxError: unexpected EOF while parsing (<ipython-input-35-136dd5165412>, line 1)

In [36]:
(a_ <= ei)

TypeError: Incompatible function arguments. The following argument types are supported:
    1. (EGA, EGA) -> EGA


In [37]:
(a_ <= ei) <= ei

TypeError: Incompatible function arguments. The following argument types are supported:
    1. (EGA, EGA) -> EGA


In [38]:
a_.comm(ei)

TypeError: Incompatible function arguments. The following argument types are supported:
    1. (EGA, EGA) -> EGA


In [39]:
vsr.Biv(1,0,0).rev() * vsr.Biv(1,0,0).rev()

Rot: [ -1 0 0 0 ]

In [40]:
Hess(a,rotor0);

TypeError: Hess() takes exactly 3 arguments (2 given)

In [41]:
J = np.array([jac(a, rotor0) for a in vectors_a]).reshape(-1,3)
g = np.array([res(a,b,rotor0) for a,b in zip(vectors_a, vectors_b)]).reshape(-1,1)
np.dot(J.T,g)

NameError: name 'jac' is not defined

In [42]:
[np.sum(np.array((a.spin(rotor0) ^ b)).reshape(-1,3), axis=0) for a,b in zip(vectors_a, vectors_b)]

[array([-0.2802,  0.1402, -0.0418]),
 array([-0.2661, -0.3822, -0.454 ]),
 array([-1.5045,  0.1896,  0.1503]),
 array([-2.1645,  2.4065,  0.8209]),
 array([-0.2103,  0.0373, -0.0497]),
 array([-0.0076, -0.0058, -0.002 ]),
 array([-1.2926, -0.9102,  0.5033]),
 array([-0.5217, -0.0341,  0.1302]),
 array([-0.3868,  0.0095,  0.1462]),
 array([-0.5735, -0.3046,  0.168 ])]

In [43]:
rotor0.rev() * (vectors_a[1].spin(rotor0) ^ vectors_b[1])

Rot: [ 0.27 -1.6e-17 -0.45 0.38 ]

In [44]:
J = np.array([jac(a, rotor0) for a in vectors_a]).reshape(-1,3)
J

NameError: name 'jac' is not defined

In [45]:
np.dot(J.T,J)

array([[ 1.2942, -0.2456, -0.2322],
       [-0.2456,  0.699 , -0.6461],
       [-0.2322, -0.6461,  0.7718]])

In [46]:
H = np.sum([Hess(a,b, rotor0) for a,b in zip(vectors_a, vectors_b)], axis=0)
H

array([[-28.832 ,  -0.7089,  -4.0662],
       [  4.5848, -12.3054,  -7.2928],
       [  5.4855,  -3.5496, -16.5266]])

In [90]:
H = np.sum([np.array(a.spin(rotor0) ^ b) for a,b in zip(vectors_a, vectors_b)], axis=0)
H

array([-7.208 ,  1.1462,  1.3714])

In [20]:
def Hess(a,b, rot):
    j = np.zeros((3,3))
    R = vsr.EGA(rot)
    a = vsr.EGA(a)
    b = vsr.EGA(b)
    for i in range(3):
        ei = vsr.Biv(0.0, 0.0, 0.0)
        ei[i] = 1.0
        ei = vsr.EGA(ei)
        j[:,i] = -2.0 * np.array(((ei * a * R.rev() * b - b * R * a * ei.rev())))[4:7]
    return j
np.sum([Hess(a,b,rotor0) for a,b in zip(vectors_a, vectors_b)], axis=0)

array([[-50.0302,   2.3872,  -8.4366],
       [ -2.3872, -50.0302,  -8.8005],
       [  8.4366,   8.8005, -50.0302]])

In [36]:
def update(vectors_a, vectors_b, rot):
#     H = -2 * np.sum([motor_jacobian.outer_product(a, b, rot)[:,1:] for a,b in zip(vectors_a, vectors_b)], axis=0)
    J = np.array([jacrot(a, rot) for a in vectors_a]).reshape(-1,3)
    g = np.array([resrot(a,b,rot) for a,b in zip(vectors_a, vectors_b)]).reshape(-1,1)
    JTg = np.dot(J.T,g)
    JTJ = np.dot(J.T,J)
    H = np.sum([Hess(a,b,rot) for a,b in zip(vectors_a, vectors_b)], axis=0)
#     print(rot)
#     print(JTg)
#     print(H-JTJ)
    B = np.dot(np.linalg.pinv(H),JTg)
#     print(B)
#     B = np.sum([np.array(a.spin(rot) ^ b) for a,b in zip(vectors_a, vectors_b)], axis=0) # steepest descent
    r = vsr.Biv(*B).exp() * rot
#     grad_norm = np.linalg.norm(JTg)
    err = np.dot(g.T,g)[0][0]
    return r, err

In [37]:
r = (vsr.Biv(1,0.001, 0.001).unit() * pi/9).exp()
r0 = vsr.Rot(1,0,0,0)
r = r0
r, err = update(vectors_a, vectors_b, r)
rs = [r0, r]
n = 1

# for i in range(4):
while err > 1e-6:
    print(n)
    r, err = update(vectors_a, vectors_b, r)
    rs.append(r)
    n += 1

print(n)
print(r)

1
2
3
4
5
6
7
8
9
10
11
12
Rot: [ 0.87 -0.5 5.8e-05 1.6e-05 ]


In [81]:
    B = np.dot(np.linalg.pinv(H),JTg)
vsr.Biv(*np.sum([np.array(a.spin(rs[0]) ^b ) for a, b, in zip(vectors_a, vectors_b)],axis=0) * -2)

Biv: [ -14.42 1.584 -1.324 ]

In [84]:
B = vsr.Biv(*np.sum([np.array(a.spin(rs[1]) ^b ) for a, b, in zip(vectors_a, vectors_b)],axis=0) * -2)

In [83]:
-2 * np.sum([np.array(a.spin(r0) ^b ) for a, b, in zip(vectors_a, vectors_b)],axis=0)

array([-14.416 ,   1.5835,  -1.3235])

In [77]:
rs

[Rot: [ 1 0 0 0 ],
 Rot: [ 0.87 -0.5 -5.3e-06 0.0022 ],
 Rot: [ 0.87 -0.5 -5.4e-07 1.2e-08 ],
 Rot: [ 0.87 -0.5 -1.4e-15 -7.7e-14 ]]

In [ ]:
np.allclose(r,rotor)

In [ ]:
rotor

# Motor

In [7]:
m0 = vsr.Mot(1,0,0,0,0,0,0,0)
motor = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
print(motor)

Mot: [ 0.87 0 -0.5 0 -0.68 -0.43 -0.18 -0.25 ]


In [8]:
n_points=3
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null() for i in range(n_points)]
points_b = [point.spin(motor) for point in points_a]

In [9]:
vsr.Pnt(1,2,3,0,0).spin(motor)

Pnt: [ -2.098 2 2.366 0 2.268 ]

In [15]:
no = vsr.Pnt(0,0,0,1,0)
D = np.zeros((3,3))
for i in range(3):
    B = vsr.Drv(0,0,0)
    B[i] = 1
    D[:,i] = np.array(B.comm(no))[:3]
print(D)

[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]


In [31]:
J1 = motor_jacobian.diff_point(motor, points_a[0])
print(J1)

[[ 1.8875  0.0483  0.6447 -1.1199 -1.7321  0.0483 -1.8875  1.1199]
 [ 0.6447  0.      0.6447 -1.1199 -1.8875 -0.0483 -1.      1.7321]
 [ 0.     -1.      0.      0.     -0.3936  0.8183  4.4734 -1.7324]
 [-3.2536  0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.    ]]


In [30]:
J2 = motor_jacobian.jacobian_exp(motor, vsr.Dll(0,0,0,0,0,0))
print(J2)

[[ 0.     0.5    0.     0.     0.     0.   ]
 [ 0.866  0.    -0.5    0.     0.     0.   ]
 [ 0.     0.866  0.     0.     0.     0.   ]
 [ 0.5    0.     0.866  0.     0.     0.   ]
 [-0.433 -0.183  0.25   0.866  0.     0.5  ]
 [ 0.683 -0.25  -0.183  0.     0.866  0.   ]
 [ 0.25   0.683  0.433 -0.5    0.     0.866]
 [-0.183  0.433 -0.683  0.     0.5    0.   ]]


In [29]:
np.dot(J1,J2)

ValueError: shapes (8,5) and (8,6) not aligned: 5 (dim 1) != 8 (dim 0)

In [42]:
B =  motor.log().biv()

In [46]:
a = points_a[0]
a

Pnt: [ 0.9487 1.123 1.122 1 1.71 ]

In [40]:
R = motor.rot()

In [45]:
vsr.Biv(1,0,0).comm(a.spin(R))

Vec: [ 1.1 0.5 0 ]

In [47]:
vsr.Dll(1,0,0,0,0,0).comm(a.spin(motor))

Pnt: [ 2.123 -0.503 0 0 0 ]

In [ ]:
vsr.Vec(1,2,3).null().spin(motor)

In [ ]:
vsr.Vec(1,2,3).spin(motor.rot())

In [32]:
def jac(a, mot):
    j = np.zeros((3,6))
    for i in range(6):
        dll = vsr.Dll(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
        dll[i] = 1.0
        j[:,i] = -2.0 * np.array(dll.comm(a.spin(mot)))[:3]
    return j

In [33]:
a = points_a[0]
b = points_b[0]
jac(a,motor)

array([[-4.2467, -4.7647, -0.    ,  2.    , -0.    , -0.    ],
       [ 1.006 , -0.    , -4.7647, -0.    ,  2.    , -0.    ],
       [-0.    ,  1.006 ,  4.2467, -0.    , -0.    ,  2.    ]])

In [ ]:
def res(a,b,mot):
    return np.array(b - a.spin(mot))[:3].reshape(-1,1)

In [ ]:
J = jac(a, m0)
r = res(a,b,m0)
B = vsr.Dll(*np.dot(J.T,r))

In [ ]:
B

In [ ]:
[vsr.Dll(0,1,0,0,0,0).comm(B) 
 for B in [vsr.Dll(*np.dot(jac(a,m0).T,res(a,b,m0))) 
           for a,b in zip(points_a, points_b)]]

In [ ]:
a.spin(m) ^ b

In [ ]:
def update(points_a, points_b, mot):
    J = np.array([jac(a, mot) for a in points_a]).reshape(-1,6)
    g = np.array([res(a,b,mot) for a,b in zip(points_a, points_b)]).reshape(-1,1)
    JTg = np.dot(J.T,g)
    print(JTg)
    print()
    H = np.dot(J.T,J)
    print(H)
    print()
    B = -np.dot(np.linalg.pinv(H),JTg)
    B = -JTg
    r = vsr.Dll(*B).exp() * mot
    grad_norm = np.linalg.norm(JTg)
    err = np.dot(g.T,g)[0][0]
    return r, err, grad_norm

In [ ]:
m = m0
res(points_a[0], points_b[0], m)
m, err, gnorm = update(points_a, points_b, m)
errs = []
# for i in range(6):
while err > 1e-6:
    m, err, gnorm = update(points_a, points_b, m)
    errs.append(err)
print(m)

In [ ]:
plt.semilogy(errs)